In [1]:
"""
Title: AI539 - Machine Learning Challenge 2022 Winter Final Project
Authur: Yun-Yi Tseng
Date: 2022/03/13

This notebook used to clean data

"""
import numpy as np
import pandas as pd

import os
import glob

import warnings
from tqdm import tqdm


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
DIR_PATH = "../data"
RAW_DICT_PATH = f"{DIR_PATH}/raw"
OUTPUT_PATH = f"{DIR_PATH}/interim"

In [3]:
def data_cleaning(file_path):

    df = pd.read_csv(file_path, encoding="big5")
    df.columns = [col.replace(" ", "") for col in df.columns]
    df = df.rename(columns={"測站": "station", "日期": "time", "測項": "items"})
    df = df.drop(0, axis=0)

    geo = pd.read_csv(f"{DIR_PATH}/external/geodata.csv")

    new_df = pd.DataFrame()
    station = df.iloc[0, 0].replace(" ", "")
    # print(df.iloc[0, 0].replace(" ", ""))
    for index, d in df.groupby("time"):
        chunk = d.iloc[:, 2:].transpose().reset_index()
        chunk.columns = chunk.iloc[0]
        chunk = chunk.drop(0)
        chunk["time"] = pd.to_datetime(index[:10] + " " + chunk.iloc[:, 0].astype(str))

        if station in geo["sitename"].unique():
            geo_station = geo.query("sitename == @station")
            chunk["station"] = geo_station["siteengname"].values[0]

            chunk["longitude"] = geo_station["twd97lon"].values[0]
            chunk["latitude"] = geo_station["twd97lat"].values[0]

        chunk = chunk.drop(chunk.columns[0], axis=1)
        new_df = pd.concat([new_df, chunk], join="outer")

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    filename = file_path.split("\\")[-1]
    new_df.to_csv(f"{OUTPUT_PATH}/{filename}", index=False)

In [4]:
for f in tqdm(glob.glob(f"{RAW_DICT_PATH}/**/*.csv", recursive=True)):
    data_cleaning(f)

 47%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                                    | 108/231 [03:25<03:53,  1.90s/it]


KeyboardInterrupt: 

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")

filenames = glob.glob(f"{OUTPUT_PATH}/*.csv")
new_csv = pd.concat([pd.read_csv(f) for f in filenames], join="outer")
new_csv.columns = [col.replace(" ", "") for col in new_csv.columns]
new_columns = [
    "station",
    "time",
    "longitude",
    "latitude",
    "AMB_TEMP",
    "CO",
    "NO",
    "NO2",
    "NOx",
    "O3",
    "PM10",
    "RAINFALL",
    "RH",
    "SO2",
    "WD_HR",
    "WIND_DIREC",
    "WIND_SPEED",
    "WS_HR",
    "CH4",
    "NMHC",
    "THC",
    "PM2.5",
]
new_csv = new_csv[new_columns]
new_csv.to_csv(f"{DIR_PATH}/processed/air_quality_2018to2020.csv", index=False)